# Example: Calculate Delta, Theta, Vega, Rho, and Gamma for Short and Long Options Contracts
In this example, we'll compute [the Greeks](https://en.wikipedia.org/wiki/en:Greeks_(finance)?variant=zh-tw) for long (short) American Call (Put) contracts.

* [The Greeks](https://en.wikipedia.org/wiki/en:Greeks_(finance)?variant=zh-tw) are quantitative measures that describe the sensitivity of an American option's price to various market factors. 
* The typical Greeks for American options are `Delta,` `Gamma,` `Theta,` `Vega,` and `Rho,` where each quantifies how the option's value changes with movements in the underlying asset price, volatility, time, and interest rates, etc.

## Setup
Set up the computational environment by including the `Include.jl` file. The `Include.jl` file loads external packages, various functions we will use in the exercise, and custom types to model the components of our example problem.
* For additional information on functions and types used in this material, see the [Julia programming language documentation](https://docs.julialang.org/en/v1/) and the [VLQuantitativeFinancePackage.jl documentation](https://github.com/varnerlab/VLQuantitativeFinancePackage.jl). 

In [3]:
include("Include.jl")

## Prerequisites
Let's build models for options on [NVIDIA Corporation](https://finance.yahoo.com/quote/NVDA/) as the underlying asset with ticker `NVDA,` in particular, out-of-the-money (OTM) short call and put options.  First, we set some constants and parameter values that will be used in the notebook, and then we'll build contract models.

### Parameters and constants
Let's set a value for the step size `Δt,` i.e., `1 day` in units of years (assuming a `365-day` year), the risk-free rate at the time we gathered the data for the option contracts; we approximate this value using the [yield on the 10-year Treasury Note](https://ycharts.com/indicators/10_year_treasury_rate). We also set a value for the underlying share price `Sₒ` and the number of levels in the binomial tree model, `h`; we assume 6-levels per day.

In [6]:
Δt = (1/365);
risk_free_rate = 0.0431; # yield 10-year treasury note on 13-Jun-2024
Sₒ = 131.88; # after close on 14-Jun-2024 (close: 131.88)
h = 366; # 6-points per trading day

### Contract models
Next, we construct contract models. We begin by building an instance of the [MyAmericanCallContractModel type](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.MyAmericanCallContractModel) using the [custom build(...) method](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.build-Tuple{Type{MyAmericanCallContractModel},%20NamedTuple}). The [build(...) method](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.build-Tuple{Type{MyAmericanCallContractModel},%20NamedTuple}) takes two arguments: the type to build, and a [NamedTuple](https://docs.julialang.org/en/v1/base/base/#Core.NamedTuple) containing the strike price `K`, the `premium` paid for the option, the days to expiration `DTE` (units: years), the `sense = 1` (long), the number of contracts in the `copy` field and the implied volatility in the `IV` field.

We save the call option contract model in the `call_option_model` variable:

In [8]:
call_option_model = build(MyAmericanCallContractModel, (
    K = 136.0, 
    premium = 9.23, # mark price
    DTE = (61.0)*Δt,
    sense = 1,
    copy = 1,
    IV = 0.4848
));

The put contract, which is an instance of the [MyAmericanPutContractModel type](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.MyAmericanPutContractModel), is constructed in a similar way to the call option. We save the put option contract model in the `put_option_model` variable:

In [10]:
put_option_model = build(MyAmericanPutContractModel, (
    K = 128.0, 
    premium = 7.78, # mark price
    DTE = (61.0)*Δt,
    sense = 1,
    copy = 1,
    IV = 0.4787
));

__Note__: In both cases, we construct long contract models, i.e., we look at option dynamics from the perspective of a contract buyer. To convert to a short perspective, i.e., from the perspective of the contract seller, we multiply the Greek values by `-1`; see below.

## Task 1: Compute the Greeks for a Long Call Option
In this task, we'll compute the Greeks for a long call option. From the buyer's viewpoint, a long call option is a substitute for owning shares. The buyer speculates that the underlying share price will go up during the contract and hopes to profit from the increase in the option contract's price. 
* However, the option contract price is a function of many different market factors, not just the share price of the underlying asset. The Greeks provide a quantitative means of computing the contract price's sensitivity to changes in market conditions.

### `Delta` for a Long Call
`Delta` measures the rate of change in the option's premium $\mathcal{P}_{\star}$ with respect to a `+1 USD/share` change in the underlying asset's price.
* We estimate $\Delta$ using the [delta(...) function](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.delta). The [delta(...) function](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.delta) takes several arguments: the `call_option_model` instance, and parameters used to compute the risk-neutral binomial share price tree, e.g., the number of tree levels `h,` the current share price  `Sₒ,` the number of days to expiration `T,` etc.
* The [delta(...) function](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.delta) and returns a value for `Delta.` We store the estimated value for `Delta` in the `delta_long_call::Float64` variable:

In [14]:
delta_long_call = let

    # initialize
    greek_value = 0.0; # initially set the greek value to 0
    contract_model = call_option_model; # alias the contract model

    # call the greek function
    greek_value = delta(contract_model, h = h,  T = contract_model.DTE, 
        σ = contract_model.IV, Sₒ = Sₒ, μ = risk_free_rate);

    # return -
    greek_value;
end

0.4920000000000009

__Summary__: For a long call, $\Delta_{c}(m)\geq{0}$ implies that the option premium $\mathcal{P}_{c}$ increases with increasing share price. This underscores the use case of long-call contracts as share replacements for underlying shares. 

### `Theta` for a Long Call
`Theta` measures the rate of change in the options premium $\mathcal{P}_{\star}$ for a `1-day` decrease in the time to maturity `T` of the contract.
* We estimate $\Theta$ using the [theta(...) function](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.theta). The [theta(...) function](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.theta) takes several arguments: the `call_option_model` instance, and parameters used to compute the risk-neutral binomial share price tree, e.g., the number of tree levels `h,` the current share price  `Sₒ,` the number of days to expiration `T.`
* The [theta(...) function](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.theta) returns a value for `Theta.` We store the estimated value for `Theta` in the `theta_long_call::Float64` variable:

In [ ]:
theta_long_call = let

    # initialize
    greek_value = 0.0; # initially set the greek value to 0
    contract_model = call_option_model; # alias the contract model

    # call the greek function
    greek_value = theta(contract_model, h = h,  T = contract_model.DTE, 
        σ = contract_model.IV, Sₒ = Sₒ, μ = risk_free_rate);

    # return -
    greek_value;
end

-0.09199999999999875

__Summary__: For a long-call, $\Theta_{c}(m)<0$ implies that the option premium $\mathcal{P}_{c}$ decreases as we approach the expiration of the contract. Thus, all else being constant, a long call will _lose value_ over time. This loss is referred to as the time-decay of the long call contract. 

### `Vega` for a Long Call
`Vega` measures the rate of change in the option's premium $\mathcal{P}_{\star}$ with respect to a `+1%` change in the underlying asset's implied volatility `IV.`
* We estimate `Vega` using the [vega(...) function](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.vega). The [vega(...) function](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.vega) takes several arguments: the `call_option_model` instance, and several parameters used to compute the risk-neutral binomial share price tree, e.g., the number of tree levels `h,` the current share price  `Sₒ,` the number of days to expiration `T,` etc.
* The [vega(...) function](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.vega) returns a value for `Vega.` We store the estimated value for vega in the `vega_long_call::Float64` variable:

In [30]:
vega_long_call = let

    # initialize
    greek_value = 0.0; # initially set the greek value to 0
    contract_model = call_option_model; # alias the contract model

    # call the Greek function -
    greek_value = vega(contract_model, h = h,  T = contract_model.DTE, 
        σ = contract_model.IV, Sₒ = Sₒ, μ = risk_free_rate);

    # return -
    greek_value;
end

0.21600000000000108

__Summary__: For the long call contract, $V_{c}(m)>0$. Thus, the option premium for the contract will increase with increasing implied volatility. In other words, as the uncertainty in the future share price increases (measured by the implied volatility), an option contract becomes more expensive. This makes sense as the option seller demands additional compensation for the increased risk. 

### Rho for a Long Call
`Rho` measures the rate of change in the options premium $\mathcal{P}_{\star}$ for a `+1%` change in the risk-free rate $\bar{r}$. 
* We estimate `Rho` using the [rho(...) function](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.rho). The [rho(...) function](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.rho) takes several arguments: the `call_option_model` instance, and several parameters used to compute the risk-neutral binomial share price tree, e.g., the number of tree levels `h,` the current share price  `Sₒ,` the number of days to expiration `T,` etc.
* The [rho(...) function](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.rho) returns a value for `Rho.` We store the estimated value for `Rho` in the `rho_long_call::Float64` variable:

In [35]:
rho_long_call = let

    # initialize
    greek_value = 0.0; # initially set the greek value to 0
    contract_model = call_option_model; # alias the contract model

    # call the Greek function -
    greek_value = rho(contract_model, h = h,  T = contract_model.DTE, 
        σ = contract_model.IV, Sₒ = Sₒ, μ = risk_free_rate);

    # return -
    greek_value;
end

0.0940000000000012

__Summary__: For a long call contract, $\rho_{c}(m)>0$. Thus, the option premium increases with increasing risk-free rate. Under normal circumstances, the risk-free rate $\bar{r}$ doesn't significantly change. However, when interest rates increase during inflationary periods, a positive `Rho` shows that option premiums will increase, all else being equal.

### Gamma for a Long Call
`Gamma` is the last common Greek we'll explore. The $\Gamma$ measures the rate of change in the `Delta` for a `+1 USD/share` change in the underlying price.
* We estimate `Gamma` using the [gamma(...) function](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.gamma). The [gamma(...) function](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.gamma) takes several arguments: the `call_option_model` instance, and several parameters used to compute the risk-neutral binomial share price tree, e.g., the number of tree levels `h,` the current share price  `Sₒ,` the number of days to expiration `T,` etc.
* The [gamma(...) function](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.gamma) returns a value for `Gamma` which we store in the `gamma_long_call::Float64` variable:

In [42]:
gamma_long_call = let

    # initialize
    greek_value = 0.0; # initially set the greek value to 0
    contract_model = call_option_model; # alias the contract model

    # call the Greek function -
    greek_value = gamma(contract_model, h = h,  T = contract_model.DTE, 
        σ = contract_model.IV, Sₒ = Sₒ, μ = risk_free_rate);

    # return -
    greek_value;
end

0.022999999999999687

__Summary__: The long call option has a $\Gamma_{c}(m)>0$, which increases the premium's sensitivity to share price movements. The gamma value indicates how sensitive an option's delta is to changes in the underlying price, with higher gamma values indicating greater delta sensitivity.

## Task 2: Compute the Greeks for a Long Put Option
In this task, we do the exact computations as above but with a put option contract. Let's begin with `Delta.`

### `Delta` for a Long Put
Like the call option, we estimate $\Delta$ using the [delta(...) function](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.delta), but in this case using the `put_option_model` instance, all other parameters held the same:

In [50]:
delta_long_put = let

    # initialize
    greek_value = 0.0; # initially set the greek value to 0
    contract_model = put_option_model; # alias the contract model

    # call the Greek function -
    greek_value = delta(contract_model, h = h,  T = contract_model.DTE, 
        σ = contract_model.IV, Sₒ = Sₒ, μ = risk_free_rate);

    # return -
    greek_value;
end

-0.3879999999999999

__Summary__: The $\Delta_{p}(m)<0$ for the long-put (opposite sign of the call). Thus, as the underlying share price increases, the value of the put option _decreases_. This is consistent with the notion that long puts make a `bearish` directional assumption, i.e., a buyer will purchase a put option if they believe the underlying asset's share price will decrease.

### Theta for Long Put
Like the call option, we estimate $\Theta$ using the [theta(...) function](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.theta), but in this case using the `put_option_model` instance, all other parameters held the same:

In [53]:
theta_long_put = let

    # initialize
    greek_value = 0.0; # initially set the greek value to 0
    contract_model = put_option_model; # alias the contract model

    # call the Greek function -
    greek_value = theta(contract_model, h = h,  T = contract_model.DTE, 
        σ = contract_model.IV, Sₒ = Sₒ, μ = risk_free_rate);

    # return -
    greek_value;
end

-0.07499999999999929

__Summary__: The $\Theta_{p}(m)<0$ for the long-put (same sign as the call option). Thus, the long-call and the long-put behave similarly in that the price of the option contract _decreases_ as the contract approaches expiration, where all else is held constant.

### Vega for Long Put
Like the call option, we estimate `Vega` using the [vega(...) function](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.vega), but in this case using the `put_option_model` instance, all other parameters held the same:

In [57]:
vega_long_put = let

    # initialize
    greek_value = 0.0; # initially set the greek value to 0
    contract_model = put_option_model; # alias the contract model

    # call the Greek function -
    greek_value = vega(contract_model, h = h,  T = contract_model.DTE, 
        σ = contract_model.IV, Sₒ = Sₒ, μ = risk_free_rate);

    # return -
    greek_value;
end

0.20700000000000074

__Summary__: For the long put contract, $V_{p}(m)>0$. Thus, the option premium for the long put (like the long call) will increase with implied volatility; as the uncertainty in the future share price increases (measured by the implied volatility), an option contract becomes more expensive (both call and puts). This makes sense as the option seller will demand additional compensation for the additional risk. 

### Rho for Long Put
Like the call option, we estimate `Rho` using the [rho(...) function](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.rho), but in this case using the `put_option_model` instance, all other parameters held the same:

In [63]:
rho_long_put = let

    # initialize
    greek_value = 0.0; # initially set the greek value to 0
    contract_model = put_option_model; # alias the contract model

    # call the Greek function -
    greek_value = rho(contract_model, h = h,  T = contract_model.DTE, 
        σ = contract_model.IV, Sₒ = Sₒ, μ = risk_free_rate);

    # return -
    greek_value;
end

-0.0829999999999993

__Summary__: For a long put contract, $\rho_{p}(m)<0$. Thus, the option premium decreases with increasing risk-free rate. Under normal circumstances, the risk-free rate $\bar{r}$ doesn't significantly change. However, when interest rates increase during inflationary periods, a negative `Rho` shows that put option premiums will decrease, all else being equal.

### Gamma for a Long Put
Finally, like the $\Gamma$ calculation for the long-call, we estimate `Gamma` for a long put using the [gamma(...) function](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.gamma), but in this case using the `put_option_model` instance, all other parameters held the same:

In [66]:
gamma_long_put = let

    # initialize
    greek_value = 0.0; # initially set the greek value to 0
    contract_model = put_option_model; # alias the contract model

    # call the Greek function -
    greek_value = gamma(contract_model, h = h,  T = contract_model.DTE, 
        σ = contract_model.IV, Sₒ = Sₒ, μ = risk_free_rate);

    # return -
    greek_value;
end

0.020000000000000462

__Summary__: The long put option has a $\Gamma_{p}(m)>0$, which increases the premium's sensitivity to share price movements, i.e., the premium becomes more responsive to changes in share price. The `Gamma` value indicates how sensitive an option's `Delta` is to changes in the underlying price, with higher `Gamma` values indicating greater `Delta` sensitivity.

## Task 3: Comparison of Long and Short Contract Greeks
In this task, let's compare the signs of various Greek values for the long call and long put contracts and then look at how to compute the short Greek values. `Unhide` the code block below to see how we constructed the Greek table for the call and put contracts.

In [78]:
let
    df = DataFrame();

    row_data = (
        greek = "Delta",
        call = delta_long_call,
        put = delta_long_put
    );
    push!(df, row_data);

    row_data = (
        greek = "Gamma",
        call = gamma_long_call,
        put = gamma_long_put
    );
    push!(df, row_data);

    row_data = (
        greek = "Theta",
        call = theta_long_call,
        put = theta_long_put
    );
    push!(df, row_data);

    row_data = (
        greek = "Vega",
        call = vega_long_call,
        put = vega_long_put
    );
    push!(df, row_data);

    row_data = (
        greek = "Rho",
        call = rho_long_call,
        put = rho_long_put
    );
    push!(df, row_data);
    pretty_table(df, tf=tf_simple)
end

========= ========= ==========
   greek      call       put 
  String   Float64   Float64 
========= ========= ==========
   Delta     0.492    -0.388
   Gamma     0.023      0.02
   Theta    -0.092    -0.075
    Vega     0.216     0.207
     Rho     0.094    -0.083
========= ========= ==========


We computed `Delta, Theta, Vega, Rho` and `Gamma` values for long put and call contacts in the previous tasks. The immediate question is, then, what about short contracts? Computing the Greek values for short contracts is straightforward: the Greek values for long and short Greek contracts must sum to `0`:

* Denote the short value of a Greek by an overbar, i.e., the short `Delta` for a Call evaluated at market conditions $m$ would be given by $\bar{\Delta}_{c}(m)$. Then, the following condition must hold (for all market states $m$ and contracts $\star$):
$$
\begin{equation}
\bar{\Delta}_{\star}(m) + \Delta_{\star}\left(m\right) = 0\quad{\star\in\left\{c,p\right\}}
\end{equation}
$$
The interconversion between long and short Greek values is a universal concept applicable to all the Greeks we have explored.

## Disclaimer and Risks
__This content is offered solely for training and informational purposes__. No offer or solicitation to buy or sell securities or derivative products or any investment or trading advice or strategy is made, given, or endorsed by the teaching team. 

__Trading involves risk__. Carefully review your financial situation before investing in securities, futures contracts, options, or commodity interests. Past performance, whether actual or indicated by historical tests of strategies, is no guarantee of future performance or success. Trading is generally inappropriate for someone with limited resources, investment or trading experience, or a low-risk tolerance.  Only risk capital that is not required for living expenses.

__You are fully responsible for any investment or trading decisions you make__. You should decide solely based on your financial circumstances, investment or trading objectives, risk tolerance, and liquidity needs.